In [1]:
%%capture
%pip install -U transformers
%pip install -U peft
%pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install networkx

In [2]:
import networkx
import pandas as pd
import json, time, os, torch
import pickle

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
    PeftConfig,
    AutoPeftModelForCausalLM
)

In [3]:
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

In [4]:
# Load base model(Zephyr-7B 7B)
bnb_config = BitsAndBytesConfig(  
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)

model_name = "gallantVN/zephyr-chat-pizza-1"
chat_tokenizer = AutoTokenizer.from_pretrained(model_name)
chat_model = AutoModelForCausalLM.from_pretrained(
    model_name,
#     low_cpu_mem_usage=True,
    return_dict=True,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,)

tokenizer_config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/510 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/675 [00:00<?, ?B/s]

...

model-00007-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/816M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/302M [00:00<?, ?B/s]

In [5]:
def get_embedding(messages):
    tokenized_chat = chat_tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=False, return_tensors="pt")
    outputs = chat_model.generate(tokenized_chat.to("cuda:0"), max_new_tokens = 128)
    return outputs.to("cpu")
#     return tokenized_chat

def prepare_data(X, y, data):
    # extract feature
    lst = [get_embedding(data["context"])]
    for k in list(data.keys())[1:4]:
        for j in range(len(data[k])):
            print(data[k][j])
            try:
                embeding = get_embedding([data[k][j]])
                lst.append(embeding)
            except:
                pass
        X.append(lst.copy())
        tmpy = [(i, i+1) for i in range(len(lst)-1)]
        if k != 'initial_inquiry':
            tmpy.append(y[-1][-1])
            tmpy.append(y[-1][-2])
            tmpy.append((0, len(lst)-len(data[k])))
        tmpy.append((len(lst)-len(data[k]), 0))
        y.append(list(set(tmpy)))
    return X, y

# Function to modify roles and keys
def modify_roles_and_keys(data):
    for key in data:
        for item in data[key]:
            if item["role"] == "Seller" or item["role"] == "seller":
                item["role"] = "user"
            elif item["role"] == "Customer" or item["role"] == "customer":
                item["role"] = "assistant"
            item["content"] = item.pop("message")
            
def refine_data(data):
    new_data = {}
    new_data["context"] = [{"role": "system", "message": "You are a friendly chatbot assistant for ordering pizza at KinPizza."}]
    for key in list(data.keys())[:3]:
        new_data[key] = data[key]
    return new_data

In [6]:
with open("/kaggle/input/pizza-data/fixed_result.json", "rb") as f:
    datas = json.load(f)

In [7]:
# refine data
lst_data = []
for i in range(len(datas)):
    data = datas[i]["dialog"]
    new_data = refine_data(data)
    modify_roles_and_keys(new_data)
    lst_data.append(new_data)

In [8]:
# extract feature
X = []
y = []
with open("/kaggle/input/graph-base-v1-100sp/data_1000.pickle", "rb") as f:
    X, y = pickle.load(f)
tmpX = []
tmpy = []
import time
start = time.time()
for i in range(3500, 4000):
    X, y = prepare_data(tmpX, tmpy, lst_data[i])
    tmpX = X
    tmpy = y
print((time.time()-start)/3600)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
2024-07-28 12:18:59.255473: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-28 12:18:59.255573: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-28 12:18:59.409306: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to r

{'role': 'assistant', 'content': "Hello! I'm looking to order a pizza from KinPizza. Can you tell me about the different types you offer?"}


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'role': 'user', 'content': 'Absolutely! We have a wide variety of pizzas to choose from. Here are some of our most popular:'}


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

{'role': 'user', 'content': 'The total cost is 259,000 VND. To confirm your order, please provide me with your specific address and phone number.'}


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'role': 'assistant', 'content': "<customer's address> and <customer's phone>."}


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'role': 'user', 'content': 'Thank you for your order! Your Large Veggie Pizza will be delivered to you shortly.'}
6.708897635009554


In [9]:
filename = "data_1000.pickle"
with open(filename, 'wb') as file:
    pickle.dump([X, y], file)

In [10]:
X[0]

[tensor([[  523, 28766,  6574, 28766, 28767,    13,  1976,   460,   264, 10131,
          10706, 10093, 13892,   354, 19815, 20727,   438, 26137, 28753, 13983,
          28723,     2, 28705,    13, 28789, 28766,  1838, 28766, 28767,    13,
          23809, 28725,   315, 28742, 28719,  6348,   297, 19815,   264, 20727,
            477, 26137, 28753, 13983, 28723,  2418,   368,  1912,   528,   767,
           4514,   302,   284,  5769,   293,   368,  2405, 28804,     2]]),
 tensor([[  523, 28766,   489, 11143, 28766, 28767,    13, 16230, 28808,   315,
          28742, 28719,  2526,   298,  1745,   264, 20727,   477, 26137, 28753,
          13983, 28723,  2418,   368,  1912,   528,   684,   272,  1581,  4514,
            368,  2405, 28804,     2, 28705,    13, 28789, 28766,   489, 11143,
          28766, 28767,    13, 15521, 25244, 28808,   816,   506,   264,  5335,
           6677,   302,   284,  5769,   293,   298,  4987,   477, 28723,  3169,
            528,  2338,   368,  1059,   813,

In [11]:
y[0]

[(0, 1), (1, 0), (1, 2)]

In [12]:
with open(filename, 'rb') as file:
    z,c = pickle.load(file)

In [13]:
len(z)

1500